In [1]:
import pandas as pd
import pickle
from utils import load_data
from scipy.sparse import coo_matrix
import seaborn as sns

In [2]:
browsing, search, sku, info = load_data('../dataset/new/')

load browsing done...
load search done...
load sku done...
load info done...


In [3]:
browsing.dropna(inplace=True)

In [4]:
browsing = browsing.sort_values(by=['session_id_hash', 'server_timestamp_epoch_ms'])

In [5]:
browsing['product_sku_hash'] = browsing['product_sku_hash'].astype('int')

In [6]:
browsing.head()

session_id_hash  product_action  product_sku_hash  \
train 16758585                0               0             42112   
      16758587                0               0             42112   
      16758589                0               0             42112   
      16758593                0               0             42112   
      16758596                0               0             31231   

                server_timestamp_epoch_ms  hashed_url  
train 16758585              1550883829964        5870  
      16758587              1550883914033        5870  
      16758589              1550883966835        5870  
      16758593              1550884064462        5870  
      16758596              1550884254839           0

In [7]:
train = browsing.loc['train', :].copy()
test = browsing.loc['test', :].copy()

In [8]:
train['item_count'] = train.groupby('session_id_hash')['product_sku_hash'].transform('count')

In [9]:
train = train[train['item_count'] >=2]

In [10]:
train.drop(columns=['item_count'], inplace=True)

In [11]:
test_pad = test.groupby('session_id_hash').tail(1).copy()
test_pad['server_timestamp_epoch_ms'] += 1

In [12]:
test_dev = pd.concat([test, test_pad])

In [13]:
test_dev = test_dev.sort_values(by=['session_id_hash', 'server_timestamp_epoch_ms'])

In [14]:
train_set = []
valid_set = []
test_set = []

In [15]:
seq_len = 25
for sess_id, item_ids in train.groupby('session_id_hash')['product_sku_hash']:
    item_ids = item_ids.tolist()
    for idx in range(1, len(item_ids)-1):
        if idx < seq_len:
            seq = item_ids[:idx] + [0] * (seq_len - idx)
        else:
            seq = item_ids[idx - seq_len:idx]
        target = item_ids[idx]
        train_set.append((sess_id, seq, target))
        
    idx = len(item_ids)-1
    if idx < seq_len:
        seq = item_ids[:idx] + [0] * (seq_len - idx)
    else:
        seq = item_ids[idx - seq_len:idx]
    target = item_ids[idx]
    valid_set.append((sess_id, seq, target))  

In [16]:
for sess_id, item_ids in test_dev.groupby('session_id_hash')['product_sku_hash']:
    item_ids = item_ids.tolist()
    for idx in range(1, len(item_ids)-1):
        if idx < seq_len:
            seq = item_ids[:idx] + [0] * (seq_len - idx)
        else:
            seq = item_ids[idx - seq_len:idx]
        target = item_ids[idx]
        train_set.append((sess_id, seq, target))
        
    idx = len(item_ids)-1
    if idx < seq_len:
        seq = item_ids[:idx] + [0] * (seq_len - idx)
    else:
        seq = item_ids[idx - seq_len:idx]
    target = item_ids[idx]
    test_set.append((sess_id, seq, target))  

In [17]:
len(train_set), len(valid_set), len(test_set)

(5691711, 1606985, 96108)

In [18]:
for name, data in zip(['train', 'valid', 'tset'], [train_set, valid_set, test_set]):
    pathname = f'../dataset/prepared/{name}.csv'
    with open(pathname, 'w') as f:
        for user_id, seq, target in data:
            f.write(','.join([str(user_id), '|'.join(map(str, seq)), str(target)]))
            f.write('\n')